In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import plotly.graph_objects as go

On présente dans ce notebook comment lire des données d'un fichier `hdf5` généré par samurai, représentant la donnée d'un champ sur un maillage cartésien non-uniforme.

In [ ]:
def import_2d_h5(filename_f5):
    file = h5py.File(filename_f5, 'r')
    mesh = file['mesh']
    points = mesh['points']
    connectivity = mesh['connectivity']
    squares = np.array([points[connectivity[i]][:, :-1]
                       for i in range(connectivity.shape[0])])
    centers = np.array([0.25*np.sum(pts, axis=0) for pts in squares])
    fields  = {
        key: mesh['fields'][key][:]
        for key in mesh['fields'].keys()
    }
    file.close()

    return squares, centers, fields

In [ ]:
class h5_2d_data:
    def __init__(self, filename_h5):
        self.squares, self.centers, self.fields = import_2d_h5(filename_h5)

    def x(self):
        return self.centers[:, 0][:]
        
    def y(self):
        return self.centers[:, 1][:]

    def z(self, field):
        return self.fields[field][:]
    
    def keys(self):
        return self.fields.keys()

    def dx(self):
        return np.max(self.squares[:,:,0], axis=1) - np.min(self.squares[:,:,0], axis=1)
        
    def dy(self):
        return np.max(self.squares[:,:,1], axis=1) - np.min(self.squares[:,:,1], axis=1)

    def dxdy(self):
        return self.dx()*self.dy()

    def integral(self, field):
        return np.sum(self.z(field)*self.dxdy())

In [ ]:
data = h5_2d_data("y_final.h5")

Visualisons les points (centre de chaque maille) de ces données

In [ ]:
plt.plot(data.x(), data.y(), "+")
plt.axis('scaled')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect('equal')

ax.tricontour(data.x(), data.y(), data.z("u_1"), levels=5, linewidths=0.5, colors='k')
ax.tricontourf(data.x(), data.y(), data.z("u_2"), levels=200, cmap="RdBu_r")

ax.plot(np.inf, np.inf, "-", linewidth=0.5, color='k', label="$b$")
ax.plot(np.inf, np.inf, "s", color='red', alpha=0.75, label="$c$")
ax.legend()
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
plt.show()

#####

fig, ax = plt.subplots()
ax.set_aspect('equal')

ax.tricontour(data.x(), data.y(), data.z("level"), levels=range(min(data.z("level")),max(data.z("level"))+1), linewidths=0.5, colors='k')
ax.tricontourf(data.x(), data.y(), data.z("u_0"), levels=200, cmap="RdBu_r")

ax.plot(np.inf, np.inf, "-", linewidth=0.5, color='k', label="levels")
ax.plot(data.x(), data.y(), "+", color="k", alpha=0.125, label="center of cells")
ax.legend()
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
plt.show()

#####

fig, ax = plt.subplots()
ax.set_aspect('equal')

min_level = min(data.z("level"))
max_level = max(data.z("level"))

for i in range(min_level, max_level+1):
    x = data.x()[data.z("level") == i]
    y = data.y()[data.z("level") == i]
    z = data.z("level")[data.z("level") == i]
    #ax.plot(x, y, "+", color=f"C{i-min_level}", label=f"level {i}")
    ax.tricontourf(x, y, z, colors=[f"C{i-min_level}"])

#ax.legend()
ax.set_xlabel("$x$")
ax.set_ylabel("$y$")
plt.show()

Pour visualiser la donnée, celle-ci doit être représentée en 3D, et matplotlib n'est pas vraiment pratique d'utilisation dans ce contexte dans un notebook. La suite des visualisation s'effectuera en utilisant plotly.

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=data.x(),
    y=data.y(),
    z=data.z('level'),
    mode='markers',
    marker=dict(
        size=3,
        color=data.z('u_1'),
        colorscale="portland",
        opacity=0.8
    )
)])
fig.update_layout(
    autosize=False,
    width=800,
    height=800
)
fig.show()

In [ ]:
fig = go.Figure(data =
    go.Heatmap(
        x=data.x(),
        y=data.y(),
        z=data.z('u_1'),
        connectgaps=True
    ))
fig.update_layout(
    autosize=False,
    width=800,
    height=800
)
fig.show()

In [ ]:
fig = go.Figure(data=[go.Scatter3d(
    x=data.x(),
    y=data.y(),
    z=data.z('u_2'),
    surfaceaxis=2,
    mode='markers',
    marker=dict(
        size=2,
        color=data.z('u_2'),
        colorscale="portland"
    )
)])
fig.update_layout(
    autosize=False,
    width=800,
    height=800
)
fig.show()

Il est difficile d'exploiter les données pour calculer une integrale dans une seule direction à cause de la forme du maillage, de même pour effectuer une interpolation. Les seules grandeurs simples à calculer sont des intégrales sur tout le domaine pour effectuer des comparaisons.